In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import json
import time


from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

In [84]:
 # Set up headless Chrome browser
options = Options()
#options.add_argument("--headless=new")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("window-size=1920,1080")
#options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

url = f"https://www.carvana.com/cars"
wait = WebDriverWait(driver, 10)

# Start on a specific page
driver.get(url)
time.sleep(5)

result = []

# Loop over pages
NUM_PAGES = 2013
for i in range(NUM_PAGES):
    print(f"--- Page {i+1} ---")
    
    #scroll_to_bottom()
    
    # Optional: you can extract data here before clicking next
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Parse vehicle links from <script type="application/ld+json">
    json_scripts = soup.find_all("script", type="application/ld+json")
    car_links = []

    for script in json_scripts:
        try:
            data = json.loads(script.string)
            if data.get("@type") == "Vehicle":
                car_links.append(data.get("offers", {}).get("url", ""))
        except Exception:
            continue
    result.extend(car_links)
    
    try:
        next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li[data-qa="next-page"]')))
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
        time.sleep(1)
        #driver.find_element(By.TAG_NAME, "body").click()  # unfocus any search box
        #time.sleep(1)
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(2)
    except Exception as e:
        print("Next button not found or not clickable:", e)
        break

driver.quit()


--- Page 1 ---
--- Page 2 ---
--- Page 3 ---
--- Page 4 ---
--- Page 5 ---
--- Page 6 ---
--- Page 7 ---
--- Page 8 ---
--- Page 9 ---
--- Page 10 ---
--- Page 11 ---
--- Page 12 ---
--- Page 13 ---
--- Page 14 ---
--- Page 15 ---
--- Page 16 ---
--- Page 17 ---
--- Page 18 ---
--- Page 19 ---
--- Page 20 ---
--- Page 21 ---
--- Page 22 ---
--- Page 23 ---
--- Page 24 ---
--- Page 25 ---
--- Page 26 ---
--- Page 27 ---
--- Page 28 ---
--- Page 29 ---
--- Page 30 ---
--- Page 31 ---
--- Page 32 ---
--- Page 33 ---
--- Page 34 ---
--- Page 35 ---
--- Page 36 ---
--- Page 37 ---
--- Page 38 ---
--- Page 39 ---
--- Page 40 ---
--- Page 41 ---
--- Page 42 ---
--- Page 43 ---
--- Page 44 ---
--- Page 45 ---
--- Page 46 ---
--- Page 47 ---
--- Page 48 ---
--- Page 49 ---
--- Page 50 ---
--- Page 51 ---
--- Page 52 ---
--- Page 53 ---
--- Page 54 ---
--- Page 55 ---
--- Page 56 ---
--- Page 57 ---
--- Page 58 ---
--- Page 59 ---
--- Page 60 ---
--- Page 61 ---
--- Page 62 ---
--- Page 63 ---
-

## Actual data collection

In [63]:
# Read URLs from txt file into a list
with open("unique_urls.txt", "r") as file:
    unique_urls = [line.strip() for line in file if line.strip()]

# Print the list
print(f'Unique urls : {len(unique_urls)}')

prev_result = pd.read_csv('first set.csv')
scraped_urls = list(prev_result[~prev_result['bodyType'].isna()]['url'])

print(f'Scraped urls : {len(scraped_urls)}')

Unique urls : 19396
Scraped urls : 4793


In [64]:
with open("scraped_urls.txt", "w") as file:
    for url in scraped_urls:
        file.write(url + "\n")

In [56]:
import random 
import pandas as pd
from tqdm import tqdm

In [57]:
# Set up headless Chrome browser
auto_check = "https://apik.carvana.io/merch/vdp/api/autocheck/v1/get?vehicleId="
column_names = ['bodyType','curbWeight','cvnaCategories','doors','drivetrainDescription','engineDescription','exteriorColor','fairMarketValue','frontHeadRoom','frontLegRoom','fuelCapacity','fuelDescription','groundClearance','highlights','horsePower','inServiceDate','inServiceMiles','installedOptions','interiorColor','inventoryType','isAutocheckReportAccidentVehicle','kbbValue','location','make','manufacturerBasicWarrantyMiles','manufacturerBasicWarrantyMonths','manufacturerDriveTrainWarrantyMiles','manufacturerDriveTrainWarrantyMonths','marketValueLoadedDateTime','mileage','model','modelId','mpgCity','mpgCombined','mpgHighway','numberOfKeys','overallHeight','overallLength','overallWidth','parentModel','previousPrice','price','priorUseTypes','purchaseType','rearHeadRoom','rearLegRoom','remainingWarrantyMiles','remainingWarrantyMonths','remainingDriveTrainWarrantyMiles','remainingDriveTrainWarrantyMonths','saleStatus','seating','standardEquipment','stockImageData','stockNumber','stores','towCapacity','transmission','trim','trunkCapacity','vehicleId','vexVdpImageData','vin','wheelbase','year']
car_data = pd.DataFrame()

for i, car_url in tqdm(enumerate(unique_urls), total=len(unique_urls), desc="Scraping car details"):

    if car_url not in scraped_urls:
        options = Options()
        #options.add_argument("--headless=new")
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_argument("window-size=1920,1080")
        #options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36")

        driver = webdriver.Chrome(options=options)

        driver.get(car_url)
        time.sleep(3)
        car_soup = BeautifulSoup(driver.page_source, "html.parser")

        auto_check_url = auto_check + car_url[-7:]
        driver.get(auto_check_url)
        time.sleep(3)
        auto_check_soup = BeautifulSoup(driver.page_source, "html.parser")

        driver.quit()

        json_scripts = car_soup.find_all("script", type="application/json")
        for script in json_scripts:
            try:
                json_data = json.loads(script.string)
                vehicle_details = json_data.get("props", {}).get("pageProps", {}).get("forProviders", {}).get("forVehicleContext", {}).get("vehicleDetails", {})
                vehicle_details['url'] = car_url
                try:
                    owner = auto_check_soup.find("span", class_="box-title-owners").find("span").text
                except AttributeError:
                    owner = "1"
                vehicle_details['owners'] = owner

                #vehicle_details['status'] = car_response.status_code
                df = pd.DataFrame([vehicle_details])
                car_data = pd.concat([car_data, df], ignore_index=True)

            except json.JSONDecodeError:
                continue

        time.sleep(random.uniform(1, 3))


Scraping car details:  25%|██▍       | 4801/19396 [11:10:17<33:57:39,  8.38s/it]


KeyboardInterrupt: 

In [58]:
car_data

,bodyType,brochureUrl,curbWeight,cvnaCategories,doors,drivetrainDescription,engineDescription,exteriorColor,fairMarketValue,frontHeadRoom,...,installedPackages,exteriorColorOEM,windowStickerUrl,bedLength,evMpgeCity,evMpgeHighway,evRange,remainingBatteryWarrantyMiles,remainingBatteryWarrantyMonths,evTaxCreditAmount
0,Sedan,https://brochure.carvana.io/hyundai/2021-accen...,2502.0,"[{'name': 'Braking & Traction', 'cvnaFeatures'...",4,FWD,"4-Cyl, 1.6 Liter",Black,12836.0,38.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Sedan,NaN,2908.0,"[{'name': 'Braking & Traction', 'cvnaFeatures'...",4,FWD,"4-Cyl, 2.0 Liter",Orange,19481.0,38.8,...,"[{'name': 'Gt-Line Premium Package', 'price': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,SUV,NaN,3916.0,"[{'name': 'Braking & Traction', 'cvnaFeatures'...",4,AWD,"4-Cyl, Turbo, 2.0 Liter",Red,26672.0,39.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SUV,NaN,5490.0,"[{'name': 'Braking & Traction', 'cvnaFeatures'...",4,RWD,"8-Cyl, EcoTec3, 5.3L",Red,48650.0,42.3,...,NaN,Cayenne Red Tintcoat,https://windowsticker.carvana.io/GeneralMotors...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SUV,https://brochure.carvana.io/jeep/2020-compass.pdf,3327.0,"[{'name': 'Braking & Traction', 'cvnaFeatures'...",4,4WD,"4-Cyl, MultiAir, PZEV, 2.4 Liter",Gray,21505.0,39.2,...,"[{'name': 'Cold Weather Group', 'price': 995, ...",Granite Crystal Metallic Clear Coat,https://windowsticker.carvana.io/Chrysler_ws_i...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,Truck,NaN,4806.0,"[{'name': 'Braking & Traction', 'cvnaFeatures'...",4,4WD,"V6, EcoBoost, TT, 2.7L",White,23685.0,40.8,...,NaN,NaN,NaN,5.5,NaN,NaN,NaN,NaN,NaN,NaN
2473,Sedan,NaN,3159.0,"[{'name': 'Braking & Traction', 'cvnaFeatures'...",4,FWD,"4-Cyl, Turbo, 1.5 Liter",Black,20475.0,39.1,...,"[{'name': 'WHEELS, 16"" ALUMINUM', 'price': -50...",Mosaic Black Metallic,https://windowsticker.carvana.io/GeneralMotors...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2474,SUV,https://brochure.carvana.io/kia/2019-sportage.pdf,3596.0,"[{'name': 'Braking & Traction', 'cvnaFeatures'...",4,FWD,"4-Cyl, GDI, 2.4 Liter",Black,12782.0,39.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2475,SUV,NaN,4585.0,"[{'name': 'Braking & Traction', 'cvnaFeatures'...",4,RWD,"6-Cyl, Turbo, 3.0 Liter",Red,29273.0,39.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
car_data.to_csv('second set.csv')

In [60]:
df3 = pd.concat([prev_result, car_data], ignore_index=True)

In [61]:
df3.to_csv('first set.csv')

In [62]:
len(df3)

4793